In [1]:
%matplotlib qt
%run funcoes.py
#%run abre_arq.py

In [2]:
lista = lista_arquivos(diretorio='./', tipo=".bin")
dados = np.array( abrir_bin(lista, len(lista)) )*(2.5/4095)

# print(lista)
resp_full = dados[0]
ecg_full = dados[1]
ppg_full = dados[2]

fa_resp = 250
fa_ecg = 1000
fa_ppg = 250

resp_full_x = np.linspace(0, (resp_full.size/fa_resp), resp_full.size)
ecg_full_x = np.linspace(0, (ecg_full.size/fa_ecg), ecg_full.size)
ppg_full_x = np.linspace(0, (ppg_full.size/fa_ppg), ppg_full.size)

#fig, ax0 = plt.subplots(1, 1)
#ax0.plot(ecg_full_x, ecg_full)
#ax0.plot(resp_full_x, resp_full)

In [3]:
resp_full_ = filtro_sinal(resp_full, 5, 15, fa_resp, ordem = 1, rp = 0.5)
ecg_full_  = filtro_sinal(ecg_full, 5, 15, fa_ecg, ordem = 1, rp = 0.5)
ppg_full_  = filtro_sinal(ppg_full, 5, 15, fa_ppg, ordem = 1, rp = 0.5)

In [4]:
# Divisao dos sinais

resp_BB = resp_full[0:90000]             #% 6 min Broad Band
resp_BB_x = resp_full_x[0:90000]
resp_Fix = resp_full[90001:150000]       #% 4 min Fix
resp_Fix_x = resp_full_x[90001:150000]
resp_pe_BB = resp_full[225001:315000]    #% Apos 5 min para equilibrio hemodin?mico, 6 min BB
resp_pe_BB_x = resp_full_x[225001:315000]
resp_pe_Fix = resp_full[315001:]         #% 4 min Fix
resp_pe_Fix_x = resp_full_x[315001:]

ecg_BB = ecg_full[0:360000]             #% 6 min Broad Band
ecg_BB_x = ecg_full_x[0:360000]
ecg_Fix = ecg_full[360000:600000]       #% 4 min Fix
ecg_Fix_x = ecg_full_x[360000:600000]
ecg_pe_BB = ecg_full[900000:1260000]    #% Apos 5 min para equilibrio hemodin?mico, 6 min BB
ecg_pe_BB_x = ecg_full_x[900000:1260000]
ecg_pe_Fix = ecg_full[1260000:]         #% 4 min Fix
ecg_pe_Fix_x = ecg_full_x[1260000:]

ppg_BB = ppg_full[0:90000]             #% 6 min Broad Band
ppg_BB_x = ppg_full_x[0:90000]
ppg_Fix = ppg_full[90001:150000]       #% 4 min Fix
ppg_Fix_x = ppg_full_x[90001:150000]
ppg_pe_BB = ppg_full[225001:315000]    #% Apos 5 min para equilibrio hemodin?mico, 6 min BB
ppg_pe_BB_x = ppg_full_x[225001:315000]
ppg_pe_Fix = ppg_full[315001:]         #% 4 min Fix
ppg_pe_Fix_x = ppg_full_x[315001:]

In [5]:
# Deteccao de picos

resp_BB_pk     = sinal_picos( resp_full[0:90000], prominence=0.5 )             #% 6 min Broad Band
resp_Fix_pk    = sinal_picos( resp_full[90001:150000], prominence=0.5 )       #% 4 min Fix
resp_pe_BB_pk  = sinal_picos( resp_full[225001:315000], prominence=0.5 )    #% Apos 5 min para equilibrio hemodin?mico, 6 min BB
resp_pe_Fix_pk = sinal_picos( resp_full[315001:], prominence=0.5 )         #% 4 min Fix

ecg_BB_pk     = sinal_picos( ecg_full[0:360000], prominence=1 )             #% 6 min Broad Band
ecg_Fix_pk    = sinal_picos( ecg_full[360000:600000], prominence=1 )       #% 4 min Fix
ecg_pe_BB_pk  = sinal_picos( ecg_full[900000:1260000], prominence=1 )    #% Apos 5 min para equilibrio hemodin?mico, 6 min BB
ecg_pe_Fix_pk = sinal_picos( ecg_full[1260000:], prominence=1 )         #% 4 min Fix

ppg_BB_pk     = sinal_picos( ppg_full[0:90000], prominence=0 )             #% 6 min Broad Band
ppg_Fix_pk    = sinal_picos( ppg_full[90001:150000], prominence=0 )       #% 4 min Fix
ppg_pe_BB_pk  = sinal_picos( ppg_full[225001:315000], prominence=0 )    #% Apos 5 min para equilibrio hemodin?mico, 6 min BB
ppg_pe_Fix_pk = sinal_picos( ppg_full[315001:], prominence=0 )         #% 4 min Fix


plot_op    = ppg_pe_Fix
plot_op_pk = ppg_pe_Fix_pk
plot_op_x  = ppg_pe_Fix_x

fig, ax1 = plt.subplots(1, 1)
ax1.plot(plot_op_x, plot_op)
ax1.plot(plot_op_x[plot_op_pk], plot_op[plot_op_pk], "x")

ax1.plot(ecg_pe_Fix_x, ecg_pe_Fix)
ax1.plot(ecg_pe_Fix_x[ecg_pe_Fix_pk], ecg_pe_Fix[ecg_pe_Fix_pk], "x")

#plt.plot(ppg_BB_x, ppg_BB)
#plt.plot(ppg_BB_x[ppg_BB_pk], ppg_BB[ppg_BB_pk], "x")

In [6]:
# Calculo da frequencia do sinal

RR_BB     = sinal_periodo(ecg_BB_x[ecg_BB_pk])
RR_Fix    = sinal_periodo(ecg_Fix_x[ecg_Fix_pk])
RR_pe_BB  = sinal_periodo(ecg_pe_BB_x[ecg_pe_BB_pk])
RR_pe_Fix = sinal_periodo(ecg_pe_Fix_x[ecg_pe_Fix_pk])

with np.errstate(divide='ignore'):
    fig, ax = plt.subplots(4, 1)
    
    ax[0].set_ylabel('bpm - BB')
    ax[0].plot( resp_BB_x, 25*resp_BB, label="Resp" )
    ax[0].plot( ecg_BB_x[ecg_BB_pk], 60/RR_BB, label="ECG" )
    ax[0].plot( ppg_BB_x[ppg_BB_pk], 60/sinal_periodo(ppg_BB_x[ppg_BB_pk]), label="PPG" )
    ax[0].legend()
    
    ax[1].set_ylabel('bpm - Fix')
    ax[1].plot( resp_Fix_x, 25*resp_Fix, label="Resp" )
    ax[1].plot( ecg_Fix_x[ecg_Fix_pk], 60/RR_Fix, label="ECG" )
    ax[1].plot( ppg_Fix_x[ppg_Fix_pk], 60/sinal_periodo(ppg_Fix_x[ppg_Fix_pk]), label="PPG" )
    ax[1].legend()
    
    ax[2].set_ylabel('bpm - pe BB')
    ax[2].plot( resp_pe_BB_x, 25*resp_pe_BB, label="Resp" )
    ax[2].plot( ecg_pe_BB_x[ecg_pe_BB_pk], 60/RR_pe_BB, label="ECG" )
    ax[2].plot( ppg_pe_BB_x[ppg_pe_BB_pk], 60/sinal_periodo(ppg_pe_BB_x[ppg_pe_BB_pk]), label="PPG" )
    ax[2].legend()
    
    ax[3].set_ylabel('bpm - pe Fix')
    ax[3].plot( resp_pe_Fix_x, 25*resp_pe_Fix )
    ax[3].plot( ecg_pe_Fix_x[ecg_pe_Fix_pk], 60/RR_pe_Fix, label="ECG" )
    ax[3].plot( ppg_pe_Fix_x[ppg_pe_Fix_pk], 60/sinal_periodo(ppg_pe_Fix_x[ppg_pe_Fix_pk]), label="PPG" )
    ax[3].legend()

In [7]:
# PWTT

pwtt_BB, pwtt_BB_x         = ppg_init(ppg_BB_x, ppg_BB, ecg_BB_x, ecg_BB)
pwtt_Fix, pwtt_Fix_X       = ppg_init(ppg_Fix_x, ppg_Fix, ecg_Fix_x, ecg_Fix)
pwtt_pe_BB, pwtt_pe_BB_x   = ppg_init(ppg_pe_BB_x, ppg_pe_BB, ecg_pe_BB_x, ecg_pe_BB)
pwtt_pe_Fix, pwtt_pe_Fix_x = ppg_init(ppg_pe_Fix_x, ppg_pe_Fix, ecg_pe_Fix_x, ecg_pe_Fix, filtro = 1)

fig, ax = plt.subplots(4, 1)

ax[0].plot( pwtt_BB_x, pwtt_BB, label = "BB" )
ax[0].legend()

ax[1].plot( pwtt_Fix_X, pwtt_Fix, label = "Fix" )
ax[1].legend()

ax[2].plot( pwtt_pe_BB_x, pwtt_pe_BB, label = "pe_BB" )
ax[2].legend()

ax[3].plot( pwtt_pe_Fix_x, pwtt_pe_Fix, label = "pe_Fix" )
ax[3].legend()

In [8]:
RR_BB_p = RR_BB[0:-2]
RR_BB_s = RR_BB[1:-1]

RR_Fix_p = RR_Fix[0:-2]
RR_Fix_s = RR_Fix[1:-1]

RR_pe_BB_p = RR_pe_BB[0:-2]
RR_pe_BB_s = RR_pe_BB[1:-1]

RR_pe_Fix_p = RR_pe_Fix[0:-2]
RR_pe_Fix_s = RR_pe_Fix[1:-1]

# poincare da variacao cardiaca

# fig, ax1 = plt.subplots(1, 1)
# ax1.plot(RR_t[:RR_t.size-1], RR_amp[:RR_t.size-1], label = 'RR')
# ax1.plot(PWTT_t[:PWTT_t.size-1], PWTT_amp[:PWTT_t.size-1], label = 'PWTT')
# ax1.legend()
# plt.show()

fig, ax = plt.subplots(2, 2)

ax[0,0].plot(RR_BB_p, RR_BB_s, '.', label="RR_BB")
ax[0,0].set_xlim(0, 1.5)
ax[0,0].set_ylim(0, 1.5)
ax[0,0].legend()

ax[1,0].plot(RR_Fix_p, RR_Fix_s, '.', label="RR_Fix")
ax[1,0].set_xlim(0, 1.5)
ax[1,0].set_ylim(0, 1.5)
ax[1,0].legend()

ax[0,1].plot(RR_pe_BB_p, RR_pe_BB_s, '.', label="RR_pe_BB")
ax[0,1].set_xlim(0, 1.5)
ax[0,1].set_ylim(0, 1.5)
ax[0,1].legend()

ax[1,1].plot(RR_pe_Fix_p, RR_pe_Fix_s, '.', label="RR_pe_Fix")
ax[1,1].set_xlim(0, 1.5)
ax[1,1].set_ylim(0, 1.5)
ax[1,1].legend()

# primeiro = PWTT_amp[0:PWTT_t.size-2]
# segundo = PWTT_amp[1:PWTT_t.size-1]

# fig, ax3 = plt.subplots(1, 1)
# ax3.plot(primeiro, segundo, '.')
# ax3.set_xlim(0, 1)
# ax3.set_ylim(0, 1)
# plt.show()

In [9]:
# poincare da variacao cardiaca

pwtt_BB_p = pwtt_BB[0:-2]
pwtt_BB_s = pwtt_BB[1:-1]

pwtt_Fix_p = pwtt_Fix[0:-2]
pwtt_Fix_s = pwtt_Fix[1:-1]

pwtt_pe_BB_p = pwtt_pe_BB[0:-2]
pwtt_pe_BB_s = pwtt_pe_BB[1:-1]

pwtt_pe_Fix_p = pwtt_pe_Fix[0:-2]
pwtt_pe_Fix_s = pwtt_pe_Fix[1:-1]

fig, ax = plt.subplots(2, 2)

ax[0,0].plot(pwtt_BB_p, pwtt_BB_s, '.', label="PWTT_BB")
ax[0,0].set_xlim(0, 1.5)
ax[0,0].set_ylim(0, 1.5)
ax[0,0].legend()

ax[1,0].plot(pwtt_Fix_p, pwtt_Fix_s, '.', label="PWTT_Fix")
ax[1,0].set_xlim(0, 1.5)
ax[1,0].set_ylim(0, 1.5)
ax[1,0].legend()

ax[0,1].plot(pwtt_pe_BB_p, pwtt_pe_BB_s, '.', label="PWTT_pe_BB")
ax[0,1].set_xlim(0, 1.5)
ax[0,1].set_ylim(0, 1.5)
ax[0,1].legend()

ax[1,1].plot(pwtt_pe_Fix_p, pwtt_pe_Fix_s, '.', label="PWTT_pe_Fix")
ax[1,1].set_xlim(0, 1.5)
ax[1,1].set_ylim(0, 1.5)
ax[1,1].legend()

In [10]:
P = 5
RR_BB_2 = RR_BB - np.mean(RR_BB)
pwtt_BB_2 = pwtt_BB - np.mean(pwtt_BB)
mx_pwtt, veta_pwtt = gera_mx_veta(pwtt_BB_2, P, 0)
mx_rr, veta_rr = gera_mx_veta(RR_BB_2, P)
N = len(RR_BB)
y = list()
MDL = list()
fig, ax = plt.subplots(2, 2)
for p in range(P):
    y.append(np.matmul(mx_rr[p], veta_rr[p]) + np.matmul(mx_pwtt[p], veta_pwtt[p]))
    
    e = RR_BB_2 - y[p]
    MDL.append(np.var(e))
    MDL[p] = N*np.log(MDL[p]) + p*np.log(N)

ax[0][0].plot(RR_BB_2)
ax[0][0].plot(y[p])
ax[1][0].plot(e)
ax[0][1].plot(MDL)



system = ([1.0], veta_pwtt[2])
t_imp, y_imp = signal.impulse(system)
ax[1][1].plot(t_imp, y_imp)

/home/diesson/anaconda3/lib/python3.7/site-packages/scipy/sparse/linalg/matfuncs.py:716: LinAlgWarning: Ill-conditioned matrix (rcond=2.58268e-17): result may not be accurate.
  return solve(Q, P)
